In [1]:
import sys
sys.path.append("/home/mori/Documents/h2o/env/h2o-env/lib/python3.7/site-packages")
sys.path.append("/home/mori/Documents/h2o/code/h2o-3/h2o-py/")

# Uplift random forest modeling - H2O-3 vs. CausalML

In [2]:
from causalml.dataset import make_uplift_classification
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

import numpy as np
import pandas as pd
from IPython.display import Image
from sklearn.model_selection import train_test_split

import h2o
from causalml.metrics import plot_gain, plot_qini, plot_lift
from causalml.metrics import auuc_score
from h2o.estimators.uplift_random_forest import H2OUpliftRandomForestEstimator

The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.


In [3]:
# Data generation
df, feature_cols = make_uplift_classification(n_samples=100000, 
                                              treatment_name=["control", "treatment"],
                                              n_classification_features=10,
                                              n_classification_informative=10)

# Rename features for easy interpretation of visualization
feature_cols_new = ['feature_%s'%(i) for i in range(len(feature_cols))]
rename_dict = {feature_cols[i]:feature_cols_new[i] for i in range(len(feature_cols))}
df = df.rename(columns=rename_dict)
feature_cols = feature_cols_new

print(df.head())

df.pivot_table(values='conversion',
               index='treatment_group_key',
               aggfunc=[np.mean, np.size],
               margins=True)

  treatment_group_key  feature_0  feature_1  feature_2  feature_3  feature_4  \
0           treatment  -0.696589  -1.531339   0.598504  -0.442362  -3.368640   
1           treatment  -1.734370  -2.894930   3.653897  -0.844279  -3.343867   
2             control  -0.438270   0.038893   2.287993   1.545793  -3.453372   
3           treatment  -1.502139   1.692400   0.210930   1.052463  -2.866955   
4             control   2.535457   0.329222   0.580543   0.694155  -1.255418   

   feature_5  feature_6  feature_7  feature_8  feature_9  conversion  \
0   0.207879   1.487930  -2.754940  -2.228405   2.049576           0   
1   4.015698   2.660818  -3.401851  -4.275002   3.593160           0   
2   4.346322   3.289635   1.703655  -1.959676   1.735490           0   
3   1.522505   1.157297   2.280189  -2.859386   0.907246           0   
4  -0.079855  -3.113221   0.017193   2.156657   1.439771           0   

   treatment_effect  
0                 0  
1                 0  
2                 0 

,mean,size
,conversion,conversion
treatment_group_key,,
control,0.49838,100000
treatment,0.50194,100000
All,0.50016,200000


In [4]:
# Split data to training and testing samples for model validation (next section)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=111)
df_train.shape, df_test.shape

((160000, 13), (40000, 13))

In [5]:
ntree = 100
max_depth = 1
metric="ChiSquared"
control_name = "control"
treatment_column = "treatment_group_key"
response_column = "conversion"

# CausalML

In [6]:
# Train CausalML uplift tree
causalml_uplift_model = UpliftRandomForestClassifier(
        n_estimators=ntree,
        max_depth=max_depth,
        evaluationFunction=metric,
        control_name=control_name,
        min_samples_leaf=10,
        min_samples_treatment=10,
        normalization=False,
        random_state=42)

causalml_uplift_model.fit(df_train[feature_cols].values,
                 treatment=df_train[treatment_column].values,
                 y=df_train[response_column].values)

#df_test = df_train
causalml_preds = causalml_uplift_model.predict(df_test.values)

KeyboardInterrupt: 

In [ ]:
# Plot uplift tree
graph = uplift_tree_plot(causalml_uplift_model.uplift_forest[0].fitted_uplift_tree, feature_cols)
Image(graph.create_png())

# H2O-3

In [6]:
h2o.init(strict_version_check=False)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 min 42 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.33.1.99999
H2O_cluster_version_age:,5 hours and 53 minutes
H2O_cluster_name:,mori
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.817 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
ntree = 10
max_depth = 1
metric="KL"

In [26]:
train_h2o = h2o.H2OFrame(df_train)
train_h2o[treatment_column] = train_h2o[treatment_column].asfactor()
train_h2o[response_column] = train_h2o[response_column].asfactor()
h2o_uplift_model = H2OUpliftRandomForestEstimator(
            ntrees=ntree,
            max_depth=max_depth-1,
            treatment_column=treatment_column,
            uplift_metric=metric,
            gainslift_bins=10,
            min_rows=10,
            nbins=1000,
            seed=42,
            sample_rate=0.80,
            auuc_type="gain",
            score_each_iteration=False)
h2o_uplift_model.train(y=response_column, x=feature_cols, training_frame=train_h2o)
h2o_uplift_model

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
upliftdrf Model Build progress: |███ (failed)


OSError: Job with key $0301c0a800ab32d4ffffffff$_8d81566a5ab0772d907d4448ee024edb failed with an exception: java.lang.AssertionError: Coldata -0.9549463941310912 out of range feature_8:-8.63160939740174--2.6878368854522705 step=0.005943772511949469 nbins=1000 actNBins=1000 isInt=0
cnt=0.000000, [-8.631609 - -8.625666], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.625666 - -8.619722], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.619722 - -8.613778], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.613778 - -8.607834], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.607834 - -8.601891], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.601891 - -8.595947], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.595947 - -8.590003], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.590003 - -8.584059], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.584059 - -8.578115], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.578115 - -8.572172], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.572172 - -8.566228], mean/var=  0.00/  0.00,
cnt=0.000000, [-8.566228 

In [9]:
test_h2o = h2o.H2OFrame(df_test)
preds_h2o = h2o_uplift_model.predict(test_h2o)

perf_h2o = h2o_uplift_model.model_performance(test_h2o)
auuc_h2o = perf_h2o.auuc()
print("H2O training metrics AUUC Gain: "+str(auuc_h2o))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
upliftdrf prediction progress: |█████████████████████████████████████████████████| (done) 100%


uplift_predict,p_y1_ct1,p_y1_ct0
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208
-0.00415972,0.49792,0.50208


In [ ]:
preds_comp = preds_h2o["uplift_predict"]
preds_comp.names = ["h2o"]
preds_comp["causal"] = h2o.H2OFrame(causalml_preds)
preds_comp["diff"] = abs(preds_comp["h2o"] - preds_comp["causal"])
preds_comp[treatment_column] = h2o.H2OFrame(df_test[treatment_column].values)
preds_comp[response_column] = h2o.H2OFrame(df_test[response_column].values)
preds_comp.summary()
    
min_diff = preds_comp["diff"].min()
max_diff = preds_comp["diff"].max()
mean_diff = preds_comp["diff"].mean(return_frame=False)[0]
print("min: %f max: %f mean: %f" % (min_diff, max_diff, mean_diff))

results = preds_comp.as_data_frame()
results = results[["h2o", "causal", response_column, treatment_column]]
mapping = {'control': 0, 'treatment': 1}
results = results.replace({treatment_column: mapping})


In [ ]:
auuc = auuc_score(results, outcome_col=response_column, treatment_col=treatment_column, normalize=False)
print("H2O AUUC:")
print(auuc["h2o"])
print("CauslML AUUC:")
print(auuc["causal"])
print("Random AUUC:")
print(auuc["Random"])

## Qini

In [ ]:
plot_qini(results, outcome_col=response_column, treatment_col=treatment_column)

In [ ]:
perf_h2o.plot_auuc(metric="qini")

n, uplift = perf_h2o.plot_auuc(metric="qini", plot=False)
print(len(n))
print(n)
print(uplift)

## Lift

In [ ]:
plot_lift(results, outcome_col=response_column, treatment_col=treatment_column)

In [ ]:
perf_h2o.plot_auuc(metric="lift")

n, uplift = perf_h2o.plot_auuc(metric="lift", plot=False)
print(n)
print(uplift)

## Gain

In [ ]:
plot_gain(results, outcome_col=response_column, treatment_col=treatment_column)

In [ ]:
perf_h2o.plot_auuc(metric="gain")

n, uplift = perf_h2o.plot_auuc(metric="gain", plot=False)
print(n)
print(uplift)